# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part V: Evaluating the Model on the Test Set

In this final part of the tutorial, we will reload the model developed in Part IV and test it on the test `CandidateSet`.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We repeat our definition of the `Spouse` `Candidate` subclass from Parts II, III, and IV.

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Loading Test `CandidateSet`

We reload the test `CandidateSet` object from the previous parts of the tutorial.

In [3]:
from snorkel.models import CandidateSet

test = session.query(CandidateSet).filter(CandidateSet.name == 'News Test Candidates').one()

## Automatically Creating Features
We also generate features for the test set.

In [4]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [5]:
%time F_test = feature_manager.update(session, test, 'Train Features', False)

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 37.9 s, sys: 999 ms, total: 38.9 s
Wall time: 40.9 s


**OR** if we've already created one, we can simply load as follows:

In [6]:
%time F_test = feature_manager.load(session, test, 'Train Features')

CPU times: user 2.39 s, sys: 181 ms, total: 2.57 s
Wall time: 2.78 s


## Reloading the Discriminative Model

In [7]:
from snorkel.learning import LogReg

disc_model = LogReg()
%time disc_model.load(session, "Discriminative Params")

CPU times: user 363 ms, sys: 35.3 ms, total: 398 ms
Wall time: 407 ms


/Users/bradenhancock/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Evaluating on the Test `CandidateSet`

First, we load the test set labels and gold candidates we made in Part III.

In [8]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()
L_test = label_manager.load(session, test, "News Gold Labels")

In [9]:
gold_test_set = session.query(CandidateSet).filter(CandidateSet.name == 'News Test Candidates').one()

In [10]:
tp, fp, tn, fn = disc_model.score(F_test, L_test, gold_test_set)

Test set size:	303
----------------------------------------
Pos. class accuracy: 0.714285714286
Neg. class accuracy: 0.986486486486
----------------------------------------
Precision:	0.555555555556
Recall:		0.714285714286
F1 Score:	0.625
----------------------------------------
TP: 5 | FP: 4 | TN: 292 | FN: 2
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.714285714286
Neg. class accuracy: 0.986486486486
Corpus Precision 0.556
Corpus Recall    0.714
Corpus F1        0.625
----------------------------------------
TP: 5 | FP: 4 | TN: 292 | FN: 2



## Viewing Examples
We can also view the results on the test `CandidateSet`,  You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [11]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part V User")
else:
    sv = None

<IPython.core.display.Javascript object>

In [12]:
sv

You've completed the introduction to Snorkel!